# Problem Statement

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [48]:
# imports
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [60]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [61]:
# set up environment
openai = OpenAI()

In [62]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [63]:
system_prompt = "You are a AI coding tutorial assistant. You will explain code snippets (don't be all theoretical you can use examples as well if you feel code is too complex you can go in detail as well and also provide alternate solutions), suggest new code if asked, and answer questions about programming."
user_prompt = f"Please explain this code - {question}"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

In [64]:
# Get gpt-4o-mini to answer, with streaming

response_stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)

for chunk in response_stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

This code snippet is using a combination of Python's `yield from`, set comprehensions, and potentially iterating over a list of dictionaries. Let's break the code down step by step to understand what it does and why it's written this way.

### Code Breakdown

1. **Set Comprehension**:
   python
   {book.get("author") for book in books if book.get("author")}
   
   - This part creates a `set` of authors.
   - It iterates over a collection called `books`, which is presumably a list of dictionaries.
   - For each `book`, it calls `book.get("author")`. This retrieves the value associated with the key `"author"` in the dictionary `book`.
   - The condition `if book.get("author")` ensures that only books which actually have an `author` (i.e., the value is not `None`, empty string, etc.) are considered for the set.

   The result of this set comprehension is a set of unique authors from the list of books.

2. **`yield from`**:
   python
   yield from { ... }
   
   - `yield from` is a keyword in Python that is used in generator functions to yield all values from an iterable.
   - When the generator function containing this line is called, it will yield each element from the set of authors, one at a time.

### Complete Understanding

So, putting it all together:
- The code defines a generator that will yield all unique authors from the list of `books` as long as they exist in each book's dictionary.
- The use of a set ensures that when yielding authors, each name appears only once, regardless of how many books might belong to the same author.

### Example

Suppose you have the following list of dictionaries:

python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": "Author 1"},
    {"title": "Book D", "author": None},
    {"title": "Book E", "author": "Author 3"},
]


Using the code in a generator function like this:

python
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

for author in get_unique_authors(books):
    print(author)


### Output
This would output:

Author 1
Author 2
Author 3


### Summary
- The code efficiently extracts unique authors from a list of book dictionaries while skipping any entries without an author.
- It uses a generator to produce the results lazily (i.e., yielding one author at a time as they are requested).

Feel free to ask if you have further questions or need variations of this code!

In [53]:
# Get Llama 3.2 to answer

In [54]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [65]:
ollama_via_openai_library = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [67]:
response = ollama_via_openai_library.chat.completions.create(model=MODEL_LLAMA, messages=messages, stream=False)
display(Markdown(response.choices[0].message.content))

**Code Analysis**

The given code snippet is a Python expression that uses the `yield from` statement with a generator expression to extract author names from a list of books. Here's a breakdown:

* `yield from`: This keyword is used in Python 3.x to specify a "yield from" clause, which delegates the production of an iteration sequence to a sub-generator or iterator.
* `{book.get("author") for book in books if book.get("author")}`: This is a generator expression that:
	+ Iterates over each `book` in the `books` list.
	+ Uses the `get()` method on each book to retrieve its "author" value. If the key does not exist, it defaults to `None`.
	+ Filters out books where the "author" value is `None` by applying an implicit filter `(book.get("author") for ...) if (book.get("author"))`.
	+ Yields the author name from each book.

**Example Walkthrough**

Suppose we have a list of books:

```python
books = [
    {"id": 1, "title": "The Great Gatsby", "author": "F. Scott Fitzgerald"},
    {"id": 2, "title": "To Kill a Mockingbird", "author": None},
    {"id": 3, "title": "Pride and Prejudice", "author": "Jane Austen"}
]
```

When we run the code snippet, it will iterate over `books`, filter out books without an author, and yield their corresponding author names.

**Output**

The output of this generator expression would be:

```python
['F. Scott Fitzgerald', 'Jane Austen']
```

This is because the second book in the example list ("To Kill a Mockingbird") does not have an author specified, so it's excluded from the results.

**Why `yield from`?**

The use of `yield from` is necessary here because we want to pass the author names from each book directly to whatever function or context that follows this generator expression. Without `yield from`, we would need to return the authors as a list, which defeats the purpose of using a generator expression in the first place.

By yielding the author values directly, we allow the receiving code to process them one by one, rather than collecting all the values into a single list at once. This is especially useful when working with large datasets or streams.